<a href="https://colab.research.google.com/github/przemek-c/robot-arm/blob/main/robot-arm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Imports
For MuJoCo, but before run, first manually choose the GPU T4 workspace

In [1]:
!pip install mujoco

# Set up GPU rendering.
from google.colab import files
import distutils.util
import os
import subprocess
if subprocess.run('nvidia-smi').returncode:
  raise RuntimeError(
      'Cannot communicate with GPU. '
      'Make sure you are using a GPU Colab runtime. '
      'Go to the Runtime menu and select Choose runtime type.')

# Add an ICD config so that glvnd can pick up the Nvidia EGL driver.
# This is usually installed as part of an Nvidia driver package, but the Colab
# kernel doesn't install its driver via APT, and as a result the ICD is missing.
# (https://github.com/NVIDIA/libglvnd/blob/master/src/EGL/icd_enumeration.md)
NVIDIA_ICD_CONFIG_PATH = '/usr/share/glvnd/egl_vendor.d/10_nvidia.json'
if not os.path.exists(NVIDIA_ICD_CONFIG_PATH):
  with open(NVIDIA_ICD_CONFIG_PATH, 'w') as f:
    f.write("""{
    "file_format_version" : "1.0.0",
    "ICD" : {
        "library_path" : "libEGL_nvidia.so.0"
    }
}
""")

# Configure MuJoCo to use the EGL rendering backend (requires GPU)
print('Setting environment variable to use GPU rendering:')
%env MUJOCO_GL=egl

# Check if installation was succesful.
try:
  print('Checking that the installation succeeded:')
  import mujoco
  mujoco.MjModel.from_xml_string('<mujoco/>')
except Exception as e:
  raise e from RuntimeError(
      'Something went wrong during installation. Check the shell output above '
      'for more information.\n'
      'If using a hosted Colab runtime, make sure you enable GPU acceleration '
      'by going to the Runtime menu and selecting "Choose runtime type".')

print('Installation successful.')

# Other imports and helper functions
import time
import itertools
import numpy as np

# Graphics and plotting.
print('Installing mediapy:')
!command -v ffmpeg >/dev/null || (apt update && apt install -y ffmpeg)
!pip install -q mediapy
import mediapy as media
import matplotlib.pyplot as plt

# More legible printing from numpy.
np.set_printoptions(precision=3, suppress=True, linewidth=100)

from IPython.display import clear_output
clear_output()


For RTB

In [3]:
import google.colab
# %pip install roboticstoolbox-python>=1.0.2
%pip install roboticstoolbox-python
# %pip install colored==1.4.4

# import numpy as np
import roboticstoolbox as rtb
from spatialmath import *
from math import pi

# %matplotlib inline
# import matplotlib.pyplot as plt
# from matplotlib import cm
# np.set_printoptions(linewidth=100, formatter={'float': lambda x: f"{x:8.4g}" if abs(x) > 1e-10 else f"{0:8.4g}"})

# %matplotlib notebook

# from IPython.display import clear_output
clear_output()

### Robot:
Test the RTB

In [4]:
L1 = rtb.DHLink(d=0.0,   alpha=   0.0,   theta=0.0, a=0.0,)
L2 = rtb.DHLink(d=0.0,   alpha= -pi/2,   theta=0.0, a=0.0)
L3 = rtb.DHLink(d=0.42,  alpha=  pi/2,   theta=0.0, a=0.0)
L4= rtb.DHLink( d=0.0,   alpha=  pi/2,   theta=0.0, a=0.0)
L5= rtb.DHLink( d=0.4,   alpha= -pi/2,   theta=0.0, a=0.0)
L6= rtb.DHLink( d=0.0,   alpha= -pi/2,   theta=0.0, a=0.0)
L7= rtb.DHLink( d=0.0,   alpha=  pi/2,   theta=0.0, a=0.0)
robot = rtb.DHRobot([L1, L2, L3, L4, L5, L6, L7])

In [ ]:
QPOS = [
  -1.700,
   0.530,
  -0.029,
  -0.660,
   0.120,
   0.860,
  -0.310
]

T = robot.fkine(QPOS)
print(T)
sol = robot.ikine_LM(T)
sol

   0.5444    0.8245   -0.154     0.0001177  
  -0.7355    0.5575    0.3849    0.2663    
   0.4032   -0.09632   0.91     -0.007111  
   0         0         0         1         



IKSolution(q=array([-2.559,  1.389,  3.113,  0.66 ,  0.12 , -0.86 ,  2.832]), success=True, iterations=11, searches=1, residual=1.3646590466750007e-10, reason='Success')

Mount Google Drive

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Image:

In [ ]:
'''
xml = """
<mujoco>
  <worldbody>
    <light name="top" pos="0 0 1"/>
    <geom name="red_box" type="box" size=".2 .2 .2" rgba="1 0 0 1"/>
    <geom name="green_sphere" pos=".2 .2 .2" size=".1" rgba="0 1 0 1"/>
  </worldbody>
</mujoco>
"""
model = mujoco.MjModel.from_xml_string(xml)
'''

model = mujoco.MjModel.from_xml_path('/content/drive/MyDrive/Colab Notebooks/kuka/iiwa14.xml')
data = mujoco.MjData(model)

renderer = mujoco.Renderer(model)
mujoco.mj_step(model, data)
renderer.update_scene(data)

media.show_image(renderer.render())

Sample video:

In [ ]:
model = mujoco.MjModel.from_xml_path('/content/drive/MyDrive/Colab Notebooks/kuka/iiwa14.xml')
data = mujoco.MjData(model)

# enable joint visualization option:
scene_option = mujoco.MjvOption()
scene_option.flags[mujoco.mjtVisFlag.mjVIS_JOINT] = True

duration = 3.8  # (seconds)
framerate = 60  # (Hz)

# Simulate and display video.
frames = []
mujoco.mj_resetData(model, data)
with mujoco.Renderer(model) as renderer:
  while data.time < duration:
    for i in range (7):
      data.qpos[i] = QPOS[i] + 0.5 * np.sin(data.time * (i + 1))  # Example dynamic update

    mujoco.mj_step(model, data)
    if len(frames) < data.time * framerate:
      renderer.update_scene(data, scene_option=scene_option)
      pixels = renderer.render()
      frames.append(pixels)

media.show_video(frames, fps=framerate)

This browser does not support the video tag.


### Video with qpos stored in table  
**Assumptions:**

* You have a table (e.g., a NumPy array) called `joint_positions` where each row represents a set of joint positions for a specific time step.
* The number of rows in `joint_positions` corresponds to the total number of simulation steps.
* The number of columns in `joint_positions` matches the number of joints in your model (7 in your case).  

**Changes:**

1. Loop Structure:  
We've replaced the `while` loop with a `for` loop that iterates over the rows of the `joint_positions` table using `range(len(joint_positions))`. This ensures we go through each pre-calculated set of joint positions.
2. QPOS Update:  
Inside the loop, we now directly assign the current row of `joint_positions` to `data.qpos` using:
 ```
data.qpos[:] = joint_positions[step]
```
Use code with caution  
 This updates all the joint positions at once with the values for the current simulation step (`step`).

**Explanation:**

* The `for` loop iterates through each row of the `joint_positions` table, representing the desired joint positions at each time step.
* `data.qpos[:] = joint_positions[step]` copies the joint positions from the current row of the table into the `data.qpos` array, updating the model's joint configuration.
* The rest of the code remains the same, simulating the model, rendering frames, and creating the video.

**Benefits:**

* Pre-calculated Control: This approach allows you to pre-calculate the entire joint trajectory using any algorithm you prefer and store it in a table.
* Flexibility: You can easily modify the trajectory by changing the values in the `joint_positions` table without altering the simulation loop.
* Reproducibility: Using a pre-defined trajectory ensures consistent and reproducible results.

I hope this helps you implement the dynamic joint position updates using your pre-calculated table! Let me know if you have any further questions or need to adjust the code for a specific scenario.

In [13]:
# So prepare data with RTB
q1 = [
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0
]

q2 = [
    -1.7,
    0.53,
    -0.029,
    -0.66,
    0.12,
    0.86,
    -0.31
]

trajectory  = rtb.tools.trajectory.jtraj(q1, q2, 1200)
joint_positions = trajectory.q
print(joint_positions[:5])

[[ 0.  0.  0.  0.  0.  0.  0.]
 [-0.  0. -0. -0.  0.  0. -0.]
 [-0.  0. -0. -0.  0.  0. -0.]
 [-0.  0. -0. -0.  0.  0. -0.]
 [-0.  0. -0. -0.  0.  0. -0.]]


In [9]:
model = mujoco.MjModel.from_xml_path('/content/drive/MyDrive/Colab Notebooks/kuka/iiwa14.xml')
data = mujoco.MjData(model)

# enable joint visualization option:
scene_option = mujoco.MjvOption()
scene_option.flags[mujoco.mjtVisFlag.mjVIS_JOINT] = True

# duration = 3.8  # (seconds)
framerate = 60  # (Hz)

# Simulate and display video.

frames = []
mujoco.mj_resetData(model, data)

with mujoco.Renderer(model) as renderer:
  for step in range(len(joint_positions)):  # Iterate through pre-calculated positions
    # Update QPOS from the table
    data.qpos[:] = joint_positions[step]  # Assign joint positions for current step

    mujoco.mj_step(model, data)

    if len(frames) < data.time * framerate:
      renderer.update_scene(data, scene_option=scene_option)
      pixels = renderer.render()
      frames.append(pixels)

media.show_video(frames, fps=framerate)

This browser does not support the video tag.
